<a href="https://colab.research.google.com/github/pranitha2144/Internship-iith/blob/Resnet/ResNet50_predictions_and_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import os
import time
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D,Dense, Dropout,Activation,Flatten

from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
!pip install split_folders

In [3]:
# very important note: Don't run this after first time
#thus making this cell as comments
'''
import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
split_folders.ratio('/content/drive/My Drive/data', output="/content/drive/My Drive/data_ouput", seed=1337, ratio=(.8, .1, .1)) # default values
'''

Copying files: 1000 files [08:16,  2.02 files/s]


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/data_ouput/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/data_ouput/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 800 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [8]:
image_input=Input(shape=[224,224,3])
model=ResNet50(input_tensor=image_input,include_top=True,weights='imagenet')

102858752/102853048 [==============================] - 1s 0us/step


In [9]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [13]:
from keras import regularizers
num_classes=4
last_layer=model.get_layer('avg_pool').output
fc_1=Dense(32,activation='relu',kernel_regularizer=regularizers.l2(0.0001))(last_layer)
dropout=Dropout(0.4)(fc_1)
out=Dense(num_classes,activation='softmax',name='output_layer')(dropout)
custom_resnet_model=Model(inputs=image_input,outputs=out)
custom_resnet_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [14]:
for layer in custom_resnet_model.layers[:-3]:
  layer.trainable=False
#custom_resnet_model.layers[-2].trainable=True
custom_resnet_model.layers[-1].trainable

True

In [15]:
custom_resnet_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [16]:
custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
t=time.time()
hist=custom_resnet_model.fit(train_generator,epochs=12,shuffle=True,validation_data=validation_generator)
print("training time is %s"%(t-time.time()))

Epoch 1/12
25/25 [==============================] - 176s 7s/step - loss: 0.9350 - accuracy: 0.6175 - val_loss: 1.4927 - val_accuracy: 0.2500
Epoch 2/12
25/25 [==============================] - 170s 7s/step - loss: 0.6362 - accuracy: 0.7613 - val_loss: 1.2615 - val_accuracy: 0.2500
Epoch 3/12
25/25 [==============================] - 169s 7s/step - loss: 0.6101 - accuracy: 0.7487 - val_loss: 1.4965 - val_accuracy: 0.2500
Epoch 4/12
25/25 [==============================] - 169s 7s/step - loss: 0.5560 - accuracy: 0.8012 - val_loss: 1.2516 - val_accuracy: 0.2500
Epoch 5/12
25/25 [==============================] - 174s 7s/step - loss: 0.5159 - accuracy: 0.7950 - val_loss: 1.7437 - val_accuracy: 0.2500
Epoch 6/12
25/25 [==============================] - 171s 7s/step - loss: 0.5073 - accuracy: 0.8037 - val_loss: 1.7028 - val_accuracy: 0.2500
Epoch 7/12
25/25 [==============================] - 171s 7s/step - loss: 0.5079 - accuracy: 0.8075 - val_loss: 1.9975 - val_accuracy: 0.2500
Epoch 8/12
25